# Lab 1 : Compute Pipeline (wgpu + WGSL) - version commentée (liée au projet Cloth)

Objectif du lab :
- construire un **compute pipeline**
- comprendre **dispatch/workgroups/threads**
- utiliser des **storage buffers** (lecture/écriture)
- connecter CPU → GPU avec **bind groups**
- récupérer un résultat GPU vers Python

Lien direct avec le projet final (Cloth Simulation) :
- le tissu = un ensemble de **vertices** (positions, vitesses, masses)
- à chaque frame : un compute shader met à jour les états (**forces → vitesses → positions**)
- exactement la même mécanique que ce lab : **buffers + bind groups + dispatch**

In [8]:
import numpy as np
import time

# Petit helper pour mesurer le temps CPU côté Python
class Timer:
    def __init__(self, msg: str):
        self.msg = msg

    def __enter__(self):
        self.start = time.perf_counter()

    def __exit__(self, exc_type, exc_value, traceback):
        print(f"{self.msg}: {time.perf_counter() - self.start:.6f} s")

## 1) Données CPU (numpy) → Buffer GPU

Dans ce lab on fait un calcul simple :
- entrée `data0[i]`
- sortie `data1[i] = data0[i] * data0[i]`

🎯 **Projet Cloth :** remplace `data0`/`data1` par des buffers :
- `positions[i]` et `velocities[i]` (read_write)
- éventuellement un buffer `springs[]` ou des buffers de paramètres.

In [9]:
# Taille du tableau
N = 1024  # petit pour le lab (rapide). Projet cloth: N = nb_vertices.
assert N % 64 == 0, "Ici on dispatch par blocs de 64 threads (workgroup_size=64)."

# Entrée CPU
data0 = np.arange(N, dtype=np.int32)

# Note: on ne crée pas data1 CPU, car la sortie sera écrite par le GPU.

## 2) Initialisation wgpu : Adapter + Device

- **Adapter** = carte/driver choisi (GPU)
- **Device** = "contexte" GPU, pour créer buffers/pipelines/commandes
- **Queue** = file d'envoi des commandes (submit)

🎯 **Projet Cloth :** même init, puis tu réutilises le device/queue toute la simulation.

In [10]:
from wgpu import gpu

# Choix de l'adapter (high-performance si possible)
adapter = gpu.request_adapter_sync(power_preference="high-performance")
device = adapter.request_device_sync()

queue = device.queue

## 3) Création des buffers GPU

Points clés :
- **BufferUsage.STORAGE** : accessible depuis le compute shader (lecture/écriture)
- **BufferUsage.COPY_DST** : permet d'envoyer des données CPU → GPU
- **BufferUsage.COPY_SRC** : permet de lire des données GPU → CPU

🎯 **Projet Cloth :**
- positions/velocities = STORAGE | COPY_DST | COPY_SRC (souvent)
- paramètres "uniform" possibles, ou storage pour grands tableaux.

In [ ]:
from wgpu import BufferUsage

# Buffer d'entrée (read-only dans le shader)
buffer0 = device.create_buffer_with_data(
    data=data0.tobytes(), # on crée et on initialise en même temps c'est quoi tobytes() ? 
    # tobytes() convertit un tableau numpy en une séquence d'octets, nécessaire pour créer un buffer GPU
    usage=BufferUsage.STORAGE | BufferUsage.COPY_SRC | BufferUsage.COPY_DST,
)

# dans un projet réel, le buffer sert à stocker des données de vertex (positions, couleurs, normales, etc.).
# c'est quoi vertex? vertex = sommet en français
# dans un projet de simulation physique (cloth, fluides, etc.),
# buffer d'entrée stocke des positions, vitesses, forces, etc.
# buffer de sortie stocke les nouvelles positions, vitesses, etc.
# utiliser des **storage buffers** (lecture/écriture) pour cela. 
# quand on utilise uniforms, on ne peut pas stocker autant de données.
# pourquoi on met buffer0 en COPY_DST? pour pouvoir uploader des données CPU->GPU plus tard.
# pourquoi on met buffer0 en entrée ? STORAGE? pour que le shader compute puisse lire les données.
# 

# Buffer de sortie (écrit par le shader)
buffer1 = device.create_buffer(
    size=data0.nbytes, # même taille que buffer0
    # pas d'initialisation, on va écrire dedans depuis le shader
    # c'est quoi nbytes?
    # nbytes donne la taille en octets du tableau numpy, nécessaire pour allouer le buffer GPU
    usage=BufferUsage.STORAGE | BufferUsage.COPY_SRC | BufferUsage.COPY_DST,
)



## 4) WGSL Compute Shader

Le shader reçoit :
- `@group(0) @binding(0)` : buffer0 (read)
- `@group(0) @binding(1)` : buffer1 (read_write)

Le thread global est donné par :
- `@builtin(global_invocation_id) gid`

Dans ce lab :
- **1 thread = 1 élément i**
- `i = gid.x`

🎯 **Projet Cloth :**
- souvent **1 thread = 1 vertex**
- `i = gid.x`
- puis tu lis/écris `positions[i]`, `velocities[i]`, etc.

In [ ]:
# On charge le shader depuis le fichier compute.wgsl fourni
# a quoi sert le shader?
# le shader compute est un programme qui s'exécute sur le GPU pour effectuer des calculs parallèles.
# il lit les données d'entrée depuis des buffers, effectue des calculs, et écrit
shader_code = open("compute_annotated.wgsl", "r", encoding="utf-8").read()
print(shader_code)

// Lab 1 — Compute shader (WGSL) — version commentée
// Objectif: data1[i] = data0[i]^2
//
// LIEN PROJET CLOTH:
// - data0/data1 seraient remplacés par positions/velocities/springs/params
// - 1 thread GPU = 1 vertex (souvent)
// - gid.x sert d'index "i" dans tes buffers

@group(0) @binding(0)
var<storage, read> data0: array<i32>;

@group(0) @binding(1)
var<storage, read_write> data1: array<i32>;

@compute @workgroup_size(64)
fn main(@builtin(global_invocation_id) gid: vec3<u32>) {
    // Index global du thread (sur l'axe X)
    let i: u32 = gid.x;

    // Sécurité: si on dispatch un peu trop, on évite d'écrire hors buffer
    // (Pour le lab on a choisi N multiple de 64 donc ça ne déclenche pas.)
    if (i >= arrayLength(&data0)) {
        return;
    }

    // Calcul jouet
    data1[i] = data0[i] * data0[i];
}



## 5) Bind Group Layout + Bind Group

- Le **BindGroupLayout** décrit *la forme* : "binding 0 = storage buffer, binding 1 = storage buffer"
- Le **BindGroup** met les *vrais buffers* derrière ces bindings.

🎯 **Projet Cloth :**
- tu auras plusieurs bindings : positions, velocities, springs, params…
- et parfois plusieurs bind groups (group(0), group(1), …) si tu veux séparer.

In [ ]:
from wgpu import ShaderStage, BufferBindingType

# Layout des ressources visibles par le shader (group=0)
bind_group_layout = device.create_bind_group_layout(
    entries=[
        {  # binding 0 -> buffer0 (read)
            "binding": 0,
            "visibility": ShaderStage.COMPUTE,
            "buffer": {"type": BufferBindingType.read_only_storage},
        },
        {  # binding 1 -> buffer1 (read_write)
            "binding": 1,
            "visibility": ShaderStage.COMPUTE,
            "buffer": {"type": BufferBindingType.storage},
        },
    ]
)

# Création du bind group (association des buffers aux bindings)
bind_group = device.create_bind_group(
    layout=bind_group_layout,
    entries=[
        {"binding": 0, "resource": {"buffer": buffer0, "offset": 0, "size": data0.nbytes}},
        {"binding": 1, "resource": {"buffer": buffer1, "offset": 0, "size": data0.nbytes}},
    ],
)

## 6) Compute Pipeline (shader module → pipeline)

- `create_shader_module` compile le WGSL
- `create_compute_pipeline` crée le pipeline avec l'entrée `entry_point="main"`
- `pipeline_layout` relie les bind groups

🎯 **Projet Cloth :**
- tu gardes la même structure
- tu changes juste les bindings et le code WGSL (forces + intégration).

In [ ]:
# Compilation WGSL -> shader module
shader_module = device.create_shader_module(code=shader_code)

# Pipeline layout (liste des bind group layouts)
# c'est quoi un pipeline layout?
# un pipeline layout définit la structure des ressources (bind groups) utilisées par le pipeline.
pipeline_layout = device.create_pipeline_layout(bind_group_layouts=[bind_group_layout])

# Compute pipeline
# cest quoi un pipeline?
# un pipeline est une configuration qui définit comment les shaders sont exécutés sur le GPU,
# incluant les ressources utilisées (buffers, textures, etc.)
pipeline = device.create_compute_pipeline(
    layout=pipeline_layout,
    compute={"module": shader_module, "entry_point": "main"},
)

## 7) Command Encoder + Compute Pass + Dispatch

Étapes typiques :
1) `create_command_encoder()`
2) `begin_compute_pass()`
3) `set_pipeline()`
4) `set_bind_group()`
5) `dispatch_workgroups()`
6) `end()` puis `queue.submit(...)`

**Calcul du dispatch**
- shader: `@workgroup_size(64)`
- donc pour traiter N éléments : `dispatch_workgroups(N/64, 1, 1)`

🎯 **Projet Cloth :**
- N = nb_vertices
- parfois tu fais plusieurs dispatch (ex: forces puis intégration) ou plusieurs passes.

In [ ]:
from wgpu import GPUCommandEncoder, GPUComputePassEncoder

# Execution du pipeline compute
with Timer("Compute Pipeline (dispatch)"):
    # Enregistrement des commandes
    command_encoder: GPUCommandEncoder = device.create_command_encoder()
    # création d'une passe compute 
    compute_pass: GPUComputePassEncoder = command_encoder.begin_compute_pass()
    # Configurer la passe compute
    compute_pass.set_pipeline(pipeline)
    # lier les ressources (buffers) au pipeline
    compute_pass.set_bind_group(0, bind_group)

    # Dispatch: N éléments, workgroup_size=64 => N/64 workgroups sur X
    compute_pass.dispatch_workgroups(N // 64, 1, 1)

    compute_pass.end()

    # Envoi au GPU
    queue.submit([command_encoder.finish()])

Compute Pipeline (dispatch): 0.123572 s


## 8) Lecture du résultat GPU → CPU

`device.queue.read_buffer(buffer1)` permet de récupérer la sortie.

🎯 **Projet Cloth :**
- en général tu ne lis pas tout à chaque frame (trop lent)
- tu lis seulement pour debug, capture, ou export.
- sinon tu affiches le tissu directement depuis les buffers GPU.

In [16]:
# Récupération CPU
out: memoryview = device.queue.read_buffer(buffer1)
result = np.frombuffer(out.cast("i"), dtype=np.int32)

# Vérification
expected = data0 * data0
print("OK ?", np.all(result == expected))
print("Extrait:", result[:10])

OK ? True
Extrait: [ 0  1  4  9 16 25 36 49 64 81]


GPU : petite puce dans la carte graphique dans le pc
API: code pour interagir (dessiner et faire de calcul)

- on fait d'abord le model 
- obj : format text avec plein de lignes
- space local centré au milieu
- on met une petite camera 
- sa position (2D sur canvas)

- un GPU fait que de trinagles
- on fait de pixel
- couleur de pixel
- positon, eclairage
- couleur final après superpositon

![image.png]
(attachment:image.png)

- gpu fait tout seul:
    - vertex: prog qui s'execute pour chaque en parrall
    - fragment shader : () il renvoi la couleur 
    - 

- WEBgl : 
- a chaque fram on renitialise tout et on recommence 
- 

nouvel API :

dessiner et faire de calcul

vertex sahder et fragment shader


on a un seul appel 
bingroup

shader: compute shader (wgsl)

on declare un ou plusieur tableau en sortie + les index avec une fonction qui prends 